In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.grid_search import GridSearchCV
from utils import *

In [11]:
train = pd.read_csv('data/train_ksmod.csv').fillna(0)
test = pd.read_csv('data/test_ksmod.csv').fillna(0)

In [12]:
sample = pd.read_csv('data/sample_submission.csv')

In [14]:
dt_stump = DecisionTreeClassifier(max_depth=4, min_samples_leaf=1)
adc = AdaBoostClassifier(
    base_estimator=dt_stump,
    learning_rate=1,
    n_estimators=400,
    algorithm="SAMME")

In [15]:
adc.fit(
    train
    .iloc[
        :, 
        list(range(train_event_start, train_event_end)) +
        list(range(train_feature_start, train_feature_end)) +
        list(range(train_resource_start, train_resource_end)) +
        list(range(train_severity_start, train_severity_end))
        ], 
    train.loc[:, 'fault_severity'])

AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
          learning_rate=1, n_estimators=400, random_state=None)

In [16]:
prob = adc.predict_proba(
    test
    .iloc[
        :, 
        list(range(train_event_start, train_event_end)) +
        list(range(train_feature_start, train_feature_end)) +
        list(range(train_resource_start, train_resource_end)) +
        list(range(train_severity_start, train_severity_end))
        ])

In [17]:
adc_prob = pd.DataFrame({
    'id': sample.loc[:, 'id'].values, 
    'predict_0': prob[:, 0],
    'predict_1': prob[:, 1],
    'predict_2': prob[:, 2]})

In [18]:
adc_prob.head()

,id,predict_0,predict_1,predict_2
0,11066,0.342144,0.338336,0.319520
1,18000,0.347924,0.328704,0.323372
2,16964,0.341857,0.338439,0.319704
3,4795,0.345635,0.334755,0.319611
4,3392,0.338786,0.337113,0.324100


In [19]:
adc_prob.to_csv('adc.csv', index=False)